In [18]:

%load_ext autoreload
%autoreload 2


import pytorch_lightning as pl
import torchinfo
import torchvision.transforms as transforms

from PlantINaturalist2021DataModule import PlantINaturalist2021DataModule
from PlantINaturalist2021FinetuneMobileNetv2 import PlantINaturalist2021FinetuneMobileNetv2, TRANSFORM2

config = {
    "model_name": PlantINaturalist2021FinetuneMobileNetv2.__name__,
    "num_classes": 250,
    "learning_rate": 0.01,
    "lr_decay_epoch_step_size": 5,
    "lr_decay_rate": 0.9,
    "num_trainable_layers": 2,
}

# sanity run
pl.Trainer(max_steps=1).fit(model=PlantINaturalist2021FinetuneMobileNetv2(config), datamodule=PlantINaturalist2021DataModule(transform=TRANSFORM2))

model = PlantINaturalist2021FinetuneMobileNetv2(config)

datamodule = PlantINaturalist2021DataModule(transform=TRANSFORM2, batch_size=128, num_workers=2, pin_memory=True, data_dir="./")

torchinfo.summary(model)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


c:\Users\Gamer2\miniconda3\envs\test-pytorch\lib\site-packages\pytorch_lightning\trainer\setup.py:175: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(

  | Name  | Type        | Params
--------------------------------------
0 | model | MobileNetV2 | 2.5 M 
--------------------------------------
1.2 M     Trainable params
1.3 M     Non-trainable params
2.5 M     Total params
10.176    Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

c:\Users\Gamer2\miniconda3\envs\test-pytorch\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


c:\Users\Gamer2\miniconda3\envs\test-pytorch\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 0:   0%|          | 1/392 [00:01<08:28,  1.30s/it, loss=5.59, v_num=31, train_acc_step=0.000, train_loss_step=5.590, train_acc_epoch=0.000, train_loss_epoch=5.590]

`Trainer.fit` stopped: `max_steps=1` reached.


Epoch 0:   0%|          | 1/392 [00:01<10:58,  1.68s/it, loss=5.59, v_num=31, train_acc_step=0.000, train_loss_step=5.590, train_acc_epoch=0.000, train_loss_epoch=5.590]


Layer (type:depth-idx)                                  Param #
PlantINaturalist2021FinetuneMobileNetv2                 --
├─MobileNetV2: 1-1                                      --
│    └─Sequential: 2-1                                  --
│    │    └─Conv2dNormActivation: 3-1                   (928)
│    │    └─InvertedResidual: 3-2                       (896)
│    │    └─InvertedResidual: 3-3                       (5,136)
│    │    └─InvertedResidual: 3-4                       (8,832)
│    │    └─InvertedResidual: 3-5                       (10,000)
│    │    └─InvertedResidual: 3-6                       (14,848)
│    │    └─InvertedResidual: 3-7                       (14,848)
│    │    └─InvertedResidual: 3-8                       (21,056)
│    │    └─InvertedResidual: 3-9                       (54,272)
│    │    └─InvertedResidual: 3-10                      (54,272)
│    │    └─InvertedResidual: 3-11                      (54,272)
│    │    └─InvertedResidual: 3-12                  

In [21]:
import wandb
import torch
from pytorch_lightning.loggers import WandbLogger


run = wandb.init(project='PlantINaturalist', config=config)


In [22]:

wandb_logger = WandbLogger()
trainer = pl.Trainer(benchmark=True, logger=wandb_logger, max_time="00:00:30:00", accelerator='gpu', devices=1)

trainer.fit(model=model, datamodule=datamodule)

c:\Users\Gamer2\miniconda3\envs\test-pytorch\lib\site-packages\pytorch_lightning\loggers\wandb.py:396: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type        | Params
--------------------------------------
0 | model | MobileNetV2 | 2.5 M 
--------------------------------------
1.2 M     Trainable params
1.3 M     Non-trainable params
2.5 M     Total params
10.176    Total estimated model params size (MB)


Epoch 57:  63%|██████▎   | 62/99 [00:20<00:12,  3.07it/s, loss=0.533, v_num=nyvs, train_acc_step=0.906, train_loss_step=0.398, val_acc_step=0.485, val_loss_step=2.280, val_acc_epoch=0.395, val_loss_epoch=3.410, train_acc_epoch=0.858, train_loss_epoch=0.526] 

Time limit reached. Elapsed time is 0:30:00. Signaling Trainer to stop.


Epoch 57:  84%|████████▍ | 83/99 [00:29<00:05,  2.83it/s, loss=0.539, v_num=nyvs, train_acc_step=0.883, train_loss_step=0.468, val_acc_step=0.515, val_loss_step=2.110, val_acc_epoch=0.392, val_loss_epoch=3.520, train_acc_epoch=0.861, train_loss_epoch=0.522]


In [23]:
torch.save(model.state_dict(), 'model.pth')
artifact = wandb.Artifact('model', type='model')
artifact.add_file('model.pth')
run.log_artifact(artifact)

run.finish()

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train_acc_epoch,▁▁▂▃▃▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇████████████
train_acc_step,▁▂▃▃▃▄▄▄▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇██████▇██
train_loss_epoch,█▇▆▅▅▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss_step,█▇▆▅▅▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▂▂▁▁▁▁▁▁▁▁▁
trainer/global_step,▁▁▁▁▁▁▁▁▁▁▃▂▂▂▄▂▂▂▂▂▂▂▂▅▂▂▂▆▂▂▂▃▃▃▃▃█▃▃▃
val_acc_epoch,▁▂▃▅▅▆▆▇▇▇▇▇▇▇█▇████████████████▇███████
val_acc_step,▁▃▄▄▄▆▅▄▇▅▆▅▅▅▆▅▇▆▆█▆▅▅▅▅▅▅█▆▅█▆▆▆▆▅▆▅█▆
val_loss_epoch,█▆▄▃▃▂▁▂▂▁▁▂▁▂▁▂▁▂▂▂▂▂▃▂▂▂▃▂▃▃▂▃▃▃▃▃▃▃▃▃
val_loss_step,█▆▄▅▄▃▄▅▁▄▃▄▃▄▃▄▂▃▄▂▄▄▅▄▅▄▄▁▄▆▂▄▅▄▄▄▄▆▂▅
epoch,57


In [25]:
next(model.model.classifier.parameters())

Parameter containing:
tensor([[-6.6358e-01,  9.9875e-02,  5.9504e-01,  ..., -1.0413e+00,
          2.7859e-01, -4.8532e-01],
        [-1.7687e-01, -2.0609e-01, -2.1675e-01,  ..., -2.8517e+00,
         -4.5052e-01, -1.7546e-03],
        [-6.7904e-01, -5.0580e-01, -4.9942e-02,  ..., -2.1573e-01,
         -1.0884e-01, -3.5382e-01],
        ...,
        [ 1.7749e-01, -8.1939e-01, -9.0712e-01,  ...,  7.3891e-03,
         -3.4834e-01, -8.5776e-01],
        [ 1.0856e-01, -1.1076e+00, -7.1631e-01,  ...,  5.9486e-01,
         -3.2481e-01,  1.9371e-01],
        [-9.7422e-01, -1.2256e+00, -6.9883e-01,  ..., -6.0347e-03,
         -7.2333e-01, -1.3784e+00]], requires_grad=True)

In [24]:
import wandb
run = wandb.init()


artifact = run.use_artifact('pasoi0stefan/PlantINaturalist/model:v10', type='model')
artifact_dir = artifact.download()


run.finish()


wandb:   1 of 1 files downloaded.  


In [27]:
model_artifact = PlantINaturalist2021FinetuneMobileNetv2(config)
model_artifact.load_state_dict(torch.load(f"{artifact_dir}/model.pth"))

<All keys matched successfully>

In [28]:
next(model_artifact.model.classifier.parameters())

Parameter containing:
tensor([[-6.6358e-01,  9.9875e-02,  5.9504e-01,  ..., -1.0413e+00,
          2.7859e-01, -4.8532e-01],
        [-1.7687e-01, -2.0609e-01, -2.1675e-01,  ..., -2.8517e+00,
         -4.5052e-01, -1.7546e-03],
        [-6.7904e-01, -5.0580e-01, -4.9942e-02,  ..., -2.1573e-01,
         -1.0884e-01, -3.5382e-01],
        ...,
        [ 1.7749e-01, -8.1939e-01, -9.0712e-01,  ...,  7.3891e-03,
         -3.4834e-01, -8.5776e-01],
        [ 1.0856e-01, -1.1076e+00, -7.1631e-01,  ...,  5.9486e-01,
         -3.2481e-01,  1.9371e-01],
        [-9.7422e-01, -1.2256e+00, -6.9883e-01,  ..., -6.0347e-03,
         -7.2333e-01, -1.3784e+00]], requires_grad=True)